In [ ]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Reshape
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils, to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [ ]:
((x_train, y_train),(x_test, y_test)) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
print(y_train.shape)
print(y_test)
x_test = x_test/255
x_train =x_train/255
x_test = tf.cast(tf.reshape(x_test, [x_test.shape[0], 784]), tf.float32)
x_train = tf.cast(tf.reshape(x_train, [x_train.shape[0], 784]), tf.float32)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
print(x_test.shape)
print(y_test.shape)


(60000,)
[7 2 1 ... 4 5 6]
(10000, 784)
(10000, 10)


In [ ]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
def modelNew():

  model = Sequential()
  model.add(Conv2D(32, (7,7), strides=(1,1), padding='same', name='conv0'))
  model.add(BatchNormalization(axis = 3, name='bn0'))
  model.add(MaxPooling2D(strides=(2,2), name = 'mp0'))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(10, activation='softmax'))

  return model

In [ ]:
def modelOld():

  X_input = Input(shape=(784))

  X = tf.reshape(X_input, [-1, 28,28,1])

  if True:
    X = Conv2D(32, (7,7), strides=(1,1), padding='same', name='conv0')(X)
    X = BatchNormalization(axis = 3, name='bn0')(X)
    X = MaxPooling2D(strides=(2,2), name = 'mp0')(X)
    X = Activation('relu')(X)

  X = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv1')(X)
  X = BatchNormalization(axis = 3, name='bn1')(X)
  X = MaxPooling2D(strides=(2,2), name = 'mp1')(X)
  X = Activation('relu')(X)

  X = Flatten()(X)

  X = Dense(100, activation='relu')(X)
  X = Dense(10, activation='softmax')(X)

  model = Model(inputs = X_input, outputs = X, name = 'Test')
  return model


In [ ]:
model = modelOld()

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.fit(x=x_train, y=y_train, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
469/469 [==============================] - 88s 188ms/step - loss: 0.1509 - accuracy: 0.9541
Epoch 2/10
469/469 [==============================] - 88s 187ms/step - loss: 0.0497 - accuracy: 0.9841
Epoch 3/10
 31/469 [>.............................] - ETA: 1:20 - loss: 0.0408 - accuracy: 0.9864

KeyboardInterrupt: ignored

In [ ]:
preds = model.evaluate(x=x_test, y=y_test, return_dict=True)

313/313 [==============================] - 5s 15ms/step - loss: 0.0469 - accuracy: 0.9840


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpo2qs0xnb/assets


INFO:tensorflow:Assets written to: /tmp/tmpo2qs0xnb/assets


In [ ]:
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)